# Laboratório 2 - Redes Multicamadas

- Moisés Botarro Ferraz Silva, 8504135
- Thales de Lima Kobosighawa,  9897884
- Victor Rozzatti Tornisiello, 9806867

# Exercício 1 - MLP e OU-Excluviso

Programe uma rede neural MLP, com o algoritmo BP, utilizando a linguagem Python, para resolver o problema do OU-EXCLUSIVO, isto é, encontrando os pesos e thresholds adequados . Use uma função logística como função de ativação e inicialize os pesos aleatórios no intervalo: (-0.1, 0.1).


## Implementação de um Multilayer Perceptron

Primeiramente, vamos implementar um Perceptron multicamadas capaz de receber um número qualquer de camadas e ajustar os seus pesos utilizando Backpropagation com o método do Gradiente (Gradient Descent).

### Layers

Uma camada será representada por uma classe, contendo duas propriedades princiais:
    - Uma matriz 'weights', onde cada linha i representa um neurônio da camada e cada coluna j, o peso do neurônio ligado à entrada j
    - Um vetor 'bias' onde cada linha i, representa o bias do neurônio i
 
Uma vez que, durante o Backpropagation, precisamos calcular os valores das variações dos pesos utilizando os pesos que geraram o erro, os novos valores serão salvos em duas propriedades: "updated_weights" e "updated_bias". Ao chamar o método *update* de uma layer, as matrizes de peso e bias serão atualizadas com os novos valores.

In [1]:
import numpy as np
import random
import math
from IPython.display import display, clear_output

random.seed(0)

# Layer represents a MLP Layer
# It has two main properties:
#      - a weigth matrix containing the weights of the layer's neurons. Each line represents a neuron and 
#        the columns represent its corresponding weights
#      - a bias vector, containing the neurons's bias
# Since during the backpropagation we need to compute the weights variation using the old ones, the 
# updated_weights and updated_bias properties store the new values until the update method is called
class Layer:
    # Create a new Layer with 'size' neurons, each one linked to 'inputs_size' inputs
    def __init__(self, size, inputs_size):
        self.size = size
        self.inputs_size = inputs_size
        self.weights = np.array([[random.uniform(-0.1, 0.1) for j in range(inputs_size)] for i in range(size)])
        self.bias = np.array([random.uniform(-0.1,0.1) for i in range(size)])
        self.updated_weights = np.copy(self.weights)
        self.updated_bias = np.copy(self.bias)
    
    # update updates the weights and bias matrices with the values stored in the updated ones
    def update(self):
        self.weights = np.copy(self.updated_weights)
        self.bias = np.copy(self.updated_bias)
        
    # description prints a layer description
    def description(self):
        print("Layer Info")
        print("Weights: \n", self.weights)
        print("Bias: \n ", self.bias)

l = Layer(2, 3)
l.description()

Layer Info
Weights: 
 [[ 0.06888437  0.05159088 -0.01588568]
 [-0.04821665  0.00225494 -0.01901317]]
Bias: 
  [ 0.05675972 -0.03933745]


### Multilayer Perceptron

#### Função de Ativação

Nossa rede irá utilizar a função logística como função de ativação. Ela é definida pela funcão **logistic** abaixo. Para poder aplicar essa função em cada elemento de um array numpy, podemos criar uma versão vetorizada utilizando o comando *np.vectorize*

Durante o backpropagation, precisamos utilizar a derivada da função logística. Ela é representada pela função **logistic_derivate**.

In [2]:
def logistic(x):
    return 1.0/(1.0+ math.exp(-x))

logistic_vec = np.vectorize(logistic)

def logistic_derivate(x):
    return x*(1.0-x)

logistic_vec(np.array([0, 0, 0]))

array([0.5, 0.5, 0.5])

Iremos definir uma classe MLP representando nossa rede.

#### Criação

Para criar uma MLP, basta usar o construtor **MLP()**. As camadas podem ser passadas para o construtor no momento da instanciação da MLP ou adicionadas depois através do método *add_layer*

#### Fast Forward

Para o cálculo da saída de uma layer com pesos representados pela matrix W e bias B, a seguinte função é aplicada:

$$ output = F(W * I + B) $$

Onde I é o vetor de entradas para a camada e F(), a função de ativação. 

Abrindo a fórmula para uma camada com 2 neurônios, os quais recebem 2 entradas e utilizando a função logística como função de ativação, obtemos:

$$  
F(
\begin{bmatrix}
w_{11}&w_{12} \\
w_{21}&w_{22}
\end{bmatrix}
*
\begin{bmatrix}
i_{1} \\
i_{2} \\
\end{bmatrix}
+
\begin{bmatrix}
b_{1} \\
b_{2} \\
\end{bmatrix}
) 
=
\begin{bmatrix}
\frac{1}{1+exp({w_{11}*i_1 + w_{12}*i_2})} \\
\frac{1}{1+exp({w_{21}*i_1 + w_{22}*i_2})} \\
\end{bmatrix}
$$

Ao calcular a saída de uma layer, atribuímos ela à entrada da layer seguinte e aplicamos o processo de forma recursiva até chegarmos ao fim da MLP.

#### Backpropagation

Para calcular o novo peso $w_i$ do neurônio $j$ na iteração (k+1), utilizamos a seguinte fórmula

$$
w_{ij}(k+1) = w_{ij}(k) - \eta\frac{\partial E(w)}{\partial w_{ij}} \bigg\rvert_{w(k)}
$$

Para o cálculo de $$ \frac{\partial E(w)}{\partial w_{ij}} $$

pode-se abrí-lo em

$$ \frac{\partial E(w)}{\partial w_{ij}} = \frac{\partial E}{\partial v_j} * \frac{\partial v_j }{\partial w_{ji}} $$

Onde $v_j$ é a soma das entradas do neuônio j mais seu bias. 

Temos que:

$$ \frac{\partial v_j }{\partial w_{ji}} = i_{ij} $$

one $i_{ij}$ é a entrada i do neurônio j.

Entretanto, para o cálculo de $$ \frac{\partial E}{\partial v_j} $$

precisamos saber se estamos na última camada ou em uma cada interna. Iremos definir uma nova variável $\delta_j$ onde 

$$ \delta_j = - \frac{\partial E}{\partial v_j} $$

##### Última Camada

Na última camada, temos acesso direto ao erro cometido pelo MLP, uma vez que podemos comparar diretamente a saída obtida com a saída esperada. Assim,

$$ \frac{\partial E(w)}{\partial v_j} = (t_j-y_j)*f' $$

onde $t_j$ e $y_j$ são, respectivamente, a saída esperada e a obtida no neurônio j da última camada. $f'$ é a derivada da funçõa de ativação. Para a função logística, temos que

$$ f' = y_j(1-y_j) $$

Portanto, chegamos à fórmula do novos peso:

$$
w_{ij}(k+1) = w_{ij}(k) + \eta \delta_j i_{ij}
$$

onde 

$$ \delta_j = (t_j-y_j)* y_j*(1-y_j) $$

##### Camadas Internas 

Nas camadas internas, não podemos comparar a saída de um neurônio com a saída esperada da rede. Como alternativa, usamos os erros cometidos pelos neurônios da camada à frente conectados à sua saída, ponderados pelos pesos das conexões. Dessa forma, para atualizar o peso do neurônio j

$$ \frac{\partial E(w)}{\partial v_j} = - \sum_k{(\delta_kw_{kj})}*f' = - <\delta,w_{j}>*y_j*(1-y_j) $$

onde k refere-se ao k neurônio da camada à frente, $\delta$ é um vetor contendo os valores de $\delta_k$ para cada neurônio da camada seguinte e $w_j$ refere-se aos pesos da saída do neurônio j.

Assim, chegamos à 

$$ w_{ij}(k+1) = w_{ij}(k) + \eta \delta_j i_{ij} $$

onde 

$$ \delta_j = <k,w_{j}>*y_j*(1-y_j) $$

#### Uso

Para utilizar a MLP para fazer predições, basta usar o método predict, passando uma lista de samples para para serem classificados.

In [3]:
class MLP:
    # MLP creation. One might pass the MLP layers as parameters or add them later using the add_layer method.
    def __init__(self, *layers):
        self.layers = list()
        for layer in layers:
            self.add_layer(layer)
       
    # add_layer adds a new layer on the MLP. It verifies whether or not the new layer is compatible with the MLP
    def add_layer(self, layer):
        # If there's already a layer in the MLP, verify if the new layer is compatible
        if len(self.layers) > 0:
            if layer.inputs_size != self.layers[-1].size:
                print("The new layer is incompatible with the MLP")
                print("Please, use a layer where each neuron has the same amount of inputs as the number" \
                     "of neurons in the MLP last layer")
        
        self.layers.append(layer)
    
    # description prints the info about the MLP layers
    def description(self):
        print("-------------------------")
        print("MLP Info:")
        for layer, i in zip(self.layers, range(len(self.layers))):
            print("--- Layer: %d ---" % i)
            layer.description()
        
    # fast_forward computes the ouput for a given input vector
    def fast_forward(self,input_v):
        # We need to store each layer input in order to perform the backpropagation
        self.inputs = list()
    
        # The input is applied in a layer weights matrix and the bias is added in the result
        # Then, the logistic function is applied to each layer neuron result
        # For a layer, we have a final output vector where each component i represents the output
        # of the neuron i
        for layer in self.layers:
            self.inputs.append(input_v)
            output = logistic_vec(layer.weights @ input_v + layer.bias)
            
            # The output of the current layer is the input of the next one
            input_v = output
        
        return output
    
    # train trains the MLP using the examples passed in the samples parameter
    # The expected output for each example must be passed in the classes parameter;
    # eta represents the MLP learning rate;
    # tol represents the error tolerance. The MLP is trained until the cumulative squared error for all example
    #     is less than the tol value
    # print_status prints the output for each example during the training phase
    def train(self, samples, classes, eta=0.5, tol=1e-2, print_status=False):
        error = 2*tol
        epoch = 0
        
        while (error > tol):
            epoch += 1
            error = 0
            
            for input_v, t in zip(samples, classes):  
                # ---- Compute the output for the given input vector ----
                output = self.fast_forward(input_v)
                
                # Compute the squared error
                error_sample = pow((np.array(t)-np.array(output)),2)
                # We need to sum the error of each component when the output is a vector
                error += sum(error_sample)
                
                if (print_status == True):
                    print("\ttraining example: %s from class %s" % (input_v, t), end = " ")
                    print("y = ", output)

                    
                # ---- Backpropagation ----
                # Compute the new weights of each layer
                # Remark: the udpated weights are stored as a layer property and the layer is updated once 
                # the backpropagation is finished
                # It's necessary to do so in order to compute the delta value for the inner layers. We need 
                # to use the weights that caused the error to compute the delta instead of the updated weights
                for l in reversed(range(len(self.layers))): # Traverse the layers in reversed order
                    layer = self.layers[l]
             
                    deltas = list()
                    # Compute the delta for each layer neuron n
                    for n in range(len(layer.weights)):
                        # Last Layer
                        if l == (len(self.layers)-1):
                            delta = (t[n]-output[n])*logistic_derivate(output[n])
                            
                        # Inner Layer
                        else:
                            # output of the current layer is the input of the next one
                            neuron_output = self.inputs[l+1][n]
                            # weights of each neuron output
                            errors_weights = self.layers[l+1].weights[:,n]
                            
                            delta = np.dot(delta_next_layer,errors_weights)*logistic_derivate(neuron_output)
                              
                        # Computes the new weights and bias for the neuron n
                        for w in range(len(layer.weights[n])):
                            layer.updated_weights[n][w] = layer.weights[n][w] + eta*delta*self.inputs[l][w]
                        layer.updated_bias[n] = layer.bias[n] + (eta*delta*1) # bias input = 1

                        # Store the neuron delta
                        deltas.append(delta)
                    
                    # The neurons' delta of the current layer will be used to compute the deltas of the 
                    # next inner layer
                    delta_next_layer = np.array(deltas)
                     
                # Once the backpropagation is finished for the current example, update all the weigths and bias
                for layer in self.layers:
                    layer.update()
            
            # End of a epoch
            if epoch%100 == 0: # Print status only after each 100 iterations 
                clear_output(wait=True)
                display("End of epoch " + str(epoch) + ". Total Error = " + str(error))
        
        # End of training         
        clear_output(wait=True)
        display("End of epoch " + str(epoch) + ". Total Error = " + str(error))
        
    # predicts gets a list of input samples and returns a list with the predicted outputs
    def predict(self, samples):
        outputs = list()
        for input_v in samples:
            outputs.append(self.fast_forward(input_v))
    
        return outputs

### Teste da MLP

Vamos testar nossa função de criação da MLP.

In [4]:
# MLP creation Test
print("Creating a valid MLP")
mlp = MLP(Layer(2,3), Layer(1,2))
mlp.description()

# An invalid MLP
print("-----------------------")
print("Creating an invalid MLP")
mlp = MLP(Layer(2,3), Layer(1,1))

Creating a valid MLP
-------------------------
MLP Info:
--- Layer: 0 ---
Layer Info
Weights: 
 [[-0.00468061  0.01667641  0.08162258]
 [ 0.00093737 -0.04363243  0.05116084]]
Bias: 
  [ 0.0236738  -0.04989873]
--- Layer: 1 ---
Layer Info
Weights: 
 [[0.08194925 0.0965571 ]]
Bias: 
  [0.06204345]
-----------------------
Creating an invalid MLP
The new layer is incompatible with the MLP
Please, use a layer where each neuron has the same amount of inputs as the numberof neurons in the MLP last layer


Vamos agora treinar a MLP para aproximá-la de uma porta XOR. Para isso, vamos utilizar o seguinte conjunto de treinamento:

In [5]:
samples = [[0,0],[0,1],[1,0],[1,1]]
classes = [[0],[1],[1],[0]]

Vamos utilizar uma MLP com uma cada interna com 2 neurônios e uma cada externa com 1 neurônio apenas.

In [6]:
mlp = MLP(Layer(2,2), Layer(1,2))
mlp.train(samples, classes, eta=0.5, tol=1e-2, print_status=False)

'End of epoch 3804. Total Error = 0.009998759091016675'

In [7]:
print(mlp.predict([[0,0], [1,0], [0,1], [1,1]]))

[array([0.04124961]), array([0.95255727]), array([0.95276964]), array([0.06142866])]


Como observado acima, a rede foi capaz de classificar corretamente os novos examplos passados à ela!

Vamos diminuir e aumentar a taxa de aprendizado para ver seu efeito no treinamento!

In [8]:
mlp = MLP(Layer(2,2), Layer(1,2))
mlp.train(samples, classes, eta=0.3, tol=1e-2, print_status=False)
print(mlp.predict([[0,0], [1,0], [0,1], [1,1]]))

'End of epoch 13947. Total Error = 0.009999026964639394'

[array([0.03735462]), array([0.95299675]), array([0.953101]), array([0.06456023])]


In [9]:
mlp = MLP(Layer(2,2), Layer(1,2))
mlp.train(samples, classes, eta=0.7, tol=1e-2, print_status=False)
print(mlp.predict([[0,0], [1,0], [0,1], [1,1]]))

'End of epoch 2273. Total Error = 0.009993149269389968'

[array([0.04201139]), array([0.95245161]), array([0.95278139]), array([0.06066028])]


Como era de se esperar, ao diminuir a taxa de aprendizado, precisamos de mais epochs para chegar ao mesmo nível de erro para os exemplos de treinamento. Ao aumentar essa taxa, diminuimos a quantidade de epochs. Entretanto, utilizar uma taxa de aprendizado muito alta pode impedir que o algoritmo chegue ao mínimo global!

Durante alguns treinamentos, pode ocorrer da rede não conseguir convergir em um tempo baixo. Analisamos o algoritmo cuidadosamente e isso ocorre devido à saturação da saída dos neurônios. Pelo fato da função logística possuir derivada próxima à zero em suas extremidades, ao atingir esse ponto, os pesos começam a ser atualizados muito lentamente.

Adicionado a isso, os cálculos computacionais apresentam erros naturais de aproximação que podem piorar esse cenário. Estávamos calculando o valor de delta da seguinte forma, por exemplo, para a camada de saída:

(t-output)\*output\*(1.0-output)

Isso leva a valores diferentes calculando-o através de

(t-output)\(output\*(1.0-output))

Preferimos deixar a versão final do código com a última opção uma vez que o valor da derivada da função logística vai ser calculado primeiramente e depois esse valor vai ser aproximado ao multiplicá-lo com o erro.

Além disso, reparamos que os valores com os quais os pesos são inicializados têm uma forte influência na convergência do treinamento. Abaixo, por exemplo, escolhemos os valores iniciais para os pesos e bias iguais aos presentes no exemplo do XOR no slide passado em aula.

In [14]:
mlp = MLP(Layer(2,2), Layer(1,2))
mlp.layers[0].weights = np.array([[0.4, 0.5 ], [ 0.8, 0.8]])
mlp.layers[0].bias = np.array([ -0.6, -0.2])

mlp.layers[1].weights = np.array([[0.4, 0.5 ]])
mlp.layers[1].bias = np.array([ -0.6])

mlp.train(samples, classes, eta=0.5, tol=1e-2, print_status=False)
print(mlp.predict(samples))

'End of epoch 2050. Total Error = 0.00999990438461254'

[array([0.05272843]), array([0.95216913]), array([0.95200198]), array([0.05085128])]


Nesse caso, o treinamento converge sempre após 2050 iterações!

Abaixo, vamos alterar a ordem dos exemplos no conjunto de treinamento e verificar como isso afeta o treinamento.

In [15]:
samples = [[0,0], [1,0], [1,1], [0,1]]
classes = [[0],[1],[0],[1]]

In [16]:
mlp = MLP(Layer(2,2), Layer(1,2))
mlp.layers[0].weights = np.array([[0.4, 0.5 ], [ 0.8, 0.8]])
mlp.layers[0].bias = np.array([ -0.6, -0.2])

mlp.layers[1].weights = np.array([[0.4, 0.5 ]])
mlp.layers[1].bias = np.array([ -0.6])

mlp.train(samples, classes, eta=0.5, tol=1e-2, print_status=False)
print(mlp.predict(samples))

'End of epoch 2006. Total Error = 0.009992129286234892'

[array([0.05263687]), array([0.95214748]), array([0.05108446]), array([0.95225332])]


Repare que o aprendizado convergiu com menos iteração que anteriormente. Isso se deve ao fato que, intercalando a classe dos exemplos apresentados à rede, evitamos a saturação dos neurônios, aumentando a velocidade de aprendrizado.

A seguir, vamos verificar como a estrutura da rede afeta seu desempenho.
Iniciemos com uma rede com 2 neurônios na saída.

In [17]:
mlp = MLP(Layer(2,2), Layer(2,2))
mlp.train([[0,0], [1,0], [1,1], [0,1]], [[1,0],[0,1],[1,0],[0,1]], eta=0.5, tol=1e-2, print_status=False)
print(mlp.predict([[0,0], [1,0], [0,1], [1,1]]))

'End of epoch 7398. Total Error = 0.009996677109062057'

[array([0.97043488, 0.02965037]), array([0.03317301, 0.96675121]), array([0.03311768, 0.96680663]), array([0.95634348, 0.04373264])]


Vamos utilizar 3 neurônios na camada interna.

In [18]:
mlp = MLP(Layer(3,2), Layer(2,3))
mlp.train([[0,0], [1,0], [1,1], [0,1]], [[1,0],[0,1],[1,0],[0,1]], eta=0.5, tol=1e-2, print_status=False)
print(mlp.predict([[0,0], [1,0], [0,1], [1,1]]))

'End of epoch 3784. Total Error = 0.009999867746803032'

[array([0.96878795, 0.03116384]), array([0.03342626, 0.966622  ]), array([0.03345741, 0.96659263]), array([0.95773502, 0.04220717])]


Com 5 neurônios na camada interna.

In [19]:
mlp = MLP(Layer(5,2), Layer(2,5))
mlp.train([[0,0], [1,0], [1,1], [0,1]], [[1,0],[0,1],[1,0],[0,1]], eta=0.5, tol=1e-2, print_status=False)
print(mlp.predict([[0,0], [1,0], [0,1], [1,1]]))

'End of epoch 2978. Total Error = 0.009997331019712289'

[array([0.97150779, 0.02848967]), array([0.03216858, 0.96779294]), array([0.03210304, 0.96794808]), array([0.95414193, 0.04584689])]


Para finalizar, utilizemos 10 neurônios na camada intermediária!

In [20]:
mlp = MLP(Layer(10,2), Layer(2,10))
mlp.train([[0,0], [1,0], [1,1], [0,1]], [[1,0],[0,1],[1,0],[0,1]], eta=0.5, tol=1e-2, print_status=False)
print(mlp.predict([[0,0], [1,0], [0,1], [1,1]]))

'End of epoch 3475. Total Error = 0.009999873956243223'

[array([0.96988825, 0.02997582]), array([0.03263652, 0.96773637]), array([0.03295547, 0.96693223]), array([0.95594639, 0.04392384])]


Embora não haja muita variação no número de iterações necessárias para a convergência do treinamento, aumentando o número de neurônios, principalmente na camada interna, a convergência ficou menos sensível à inicialização dos pesos e bias!!

### Solução Final

Para o problema do XOR, vamos utilizar uma rede com 5 neurônios na camada interna e 2 neurônios na última camada. Se o valor obtido pelo primeiro neurônio da última camada for maior que o segundo, o exemplo será classificado como 0. Caso contrário, trata-se de um 1.

In [6]:
random.seed(0)

samples = [[0,0], [1,0], [1,1], [0,1]]
classes = [[1,0],[0,1],[1,0],[0,1]]

mlp = MLP(Layer(5,2), Layer(2,5))
mlp.train([[0,0], [1,0], [1,1], [0,1]], [[1,0],[0,1],[1,0],[0,1]], eta=0.5, tol=1e-2, print_status=False)
predicted = mlp.predict(samples)

'End of epoch 3427. Total Error = 0.009996688438674548'

In [22]:
def get_result(x):
    if x[0] > x[1]:
        return 0
    else:
        return 1

In [23]:
for i in range(len(predicted)):
    print("sample: %s \texpected: %d \tpredicted:%d" % \
          (samples[i], get_result(classes[i]), get_result(predicted[i])))   

sample: [0, 0] 	expected: 0 	predicted:0
sample: [1, 0] 	expected: 1 	predicted:1
sample: [1, 1] 	expected: 0 	predicted:0
sample: [0, 1] 	expected: 1 	predicted:1


# Exercício 2 - Auto-Associador

Considere o problema de auto-associador (encoding problem) no qual um conjunto de padrões ortogonais de entrada são mapeados num conjunto de padrões de saída ortogonais através de uma camada oculta com um número pequeno de neurônios.

Essencialmente, o problema é aprender uma codificação de padrão com p-bits em um padrão de log2 p-bits, e em seguida aprender a decodificar esta representação num padrão de saída.

Pede-se: Construir o mapeamento gerado por uma rede multi-camadas com o algoritmo backpropagation (BP), para o caso do mapeamento identidade, considerando dois casos:


## a) Padrão de entrada e Padrão de Saída: Id(8x8) e Id(8X8), onde Id denota a matriz identidade.

### Construção da Rede e Treinamento

In [7]:
N = 8
logN = math.ceil(math.log2(N))
mlp = MLP(Layer(logN, N), Layer(N,logN))

In [8]:
samples = [
    [1,0,0,0,0,0,0,0],
    [0,1,0,0,0,0,0,0],
    [0,0,1,0,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,1,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,1]
]

classes = [
    [1,0,0,0,0,0,0,0],
    [0,1,0,0,0,0,0,0],
    [0,0,1,0,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,1,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,1]
]

In [9]:
mlp.train(samples, classes)

'End of epoch 19219. Total Error = 0.0099998663620438'

### Uso da Rede

In [27]:
predicted = mlp.predict(samples)

Iremos definir uma função para imprimir os exemplos classificados. Caso haja um valor maior ou igual à 0,5 em um neurônio de saída, imprimimos 1. Caso contrário, 0 é impresso.

In [29]:
def print_decoded(decoded):
    for d in decoded:
        if d >= 0.5:
            print("1", end=" ")
        else:
            print("0", end=" ")
    print()

In [30]:
for output in predicted:
    print_decoded(output)

1 0 0 0 0 0 0 0 
0 1 0 0 0 0 0 0 
0 0 1 0 0 0 0 0 
0 0 0 1 0 0 0 0 
0 0 0 0 1 0 0 0 
0 0 0 0 0 1 0 0 
0 0 0 0 0 0 1 0 
0 0 0 0 0 0 0 1 


Como esperávamos, obtivemos na saída a mesma matriz identidade utilizada como entrada!!

## b) Padrão de entrada e Padrão de Saída: Id(15x15) e Id(15X15), onde Id denota a matriz identidade.

### Construção da Rede e Treinamento

In [31]:
N = 15
logN = math.ceil(math.log2(N))
mlp = MLP(Layer(logN, N), Layer(N,logN))

Vamos construir uma matriz identidade com o auxílio do Python!

In [32]:
samples = np.zeros((N,N))
for i in range(N):
    samples[i][i] = 1
samples

classes = np.copy(samples)

In [33]:
print("Samples:")
print(samples)

Samples:
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [34]:
print("Classes: ")
print(classes)

Classes: 
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [35]:
mlp.train(samples, classes)

'End of epoch 39644. Total Error = 0.009999896097870189'

### Uso da Rede

In [36]:
predicted = mlp.predict(samples)

In [37]:
for output in predicted:
    print_decoded(output)

1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 


Novamente, obtivemos como saída a mesma matriz identidade usada como entrada! Entretanto, é interessante notar que o número de iterações necessárias para a convergência do treinamento com a entrada 15x15 foi maior que o utilizado para o treinamento da rede menor!